In [29]:
import pandas as pd
import numpy as np
import random
import sys
sys.path.append('..')
from modules import utils

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

#### Useful functions

In [3]:
def create_label(row):
    '''
    Function to label the dataset
    '''
    if row['hemoglobin']> 13:
        return 'No anemia'
    elif (row['hemoglobin'] > 12) & (row['gender']==0):
        return 'No anemia'
    else:
        if row['mcv']<80 :
            if row['ferritin']<30:
                return 'Iron deficiency anemia'
            elif row['ferritin']>100:
                return 'Anemia of chronic disease'
            elif ((row['ferritin']>=30 ) & (row['tibc']<450)):
                return 'Anemia of chronic disease'
            elif ((row['ferritin']>=30) & (row['tibc']>=450)):
                return 'Iron deficiency anemia'
            else:
                return 'Inconclusive diagnosis'
            
        elif row['mcv']<=100:
            if row['ret_count'] <= 2:
                return 'Aplastic anemia'
            elif row['ret_count'] >2:
                return 'Hemolytic anemia'
            else:
                return 'Inconclusive diagnosis'

        elif row['mcv']> 100:
            if row['segmented_neutrophils']>0:
                return 'Vitamin B12/Folate deficiency anemia'
            elif row['segmented_neutrophils']==0:
                return 'Unspecified anemia'
            else:
                return 'Inconclusive diagnosis'
        else:
            return 'Inconclusive diagnosis'

In [4]:
def get_feature_dict(col_name, feat_arr):
    '''
    Returns the dictionary of a specific feature
    '''
    return [i for i in feat_arr if i['name']==col_name][0]

In [5]:
def uniform_dist(df, feat_dict, num=None):
    '''
    Generates values using a uniform distribution
    '''
    col_name = feat_dict['name']
    col_values = np.random.uniform(feat_dict['abnorm_lower'], feat_dict['abnorm_upper'], num)
    return col_values

In [6]:
def populate_columns(df, col_list, feat_arr, sample_num=None):
    '''
    Populates dataframe with values
    '''
    for col in col_list:
        feat_dict = get_feature_dict(col, feat_arr)
        df[col] = uniform_dist(df, feat_dict, sample_num)
    return df    

In [7]:
def add_nans(df, exclude_col_list=[]):
    '''
    Adding missing values to the data
    '''
    for col in df.columns:
        if col not in exclude_col_list:
            col_frac = round(random.uniform(0.1, 0.7), 1)
            df.loc[df.sample(frac=col_frac, random_state=SEED).index, col] = np.nan
    return df

In [8]:
anem_feat_arr = [{'name':'hemoglobin', 'lower':12, 'upper':17.2, 'abnorm_lower':6, 'abnorm_upper':18},
                 {'name':'ferritin', 'lower':10, 'upper':263, 'abnorm_lower':0, 'abnorm_upper': 500},
                 {'name':'ret_count', 'lower':0.5, 'upper':2.5, 'abnorm_lower':0, 'abnorm_upper':6},
                 {'name':'segmented_neutrophils', 'lower':0, 'upper':0, 'abnorm_lower':0.1, 'abnorm_upper':7},
                 {'name':'tibc', 'lower':250, 'upper':450, 'abnorm_lower': 100, 'abnorm_upper':520},
                 {'name':'mcv', 'lower':80, 'upper':100, 'abnorm_lower': 75, 'abnorm_upper':105},
                 {'name':'gender', 'lower':0, 'upper':1}, #0=Female 1=male
                ]

#### 0 - No anemia

In [9]:
no_df = pd.DataFrame()
hb_dict = get_feature_dict('hemoglobin', anem_feat_arr)
gender_dict = get_feature_dict('gender', anem_feat_arr)
no_df['hemoglobin'] = np.random.uniform(hb_dict['lower'], hb_dict['upper'], 10000)
no_df['gender'] = np.random.choice([gender_dict['lower'], gender_dict['upper']], 10000)
no_df.loc[no_df.hemoglobin <= 13, 'gender'] = 0
cols_to_populate = ['ferritin', 'ret_count', 'segmented_neutrophils', 'tibc', 'mcv']
no_df = populate_columns(no_df, cols_to_populate, anem_feat_arr, len(no_df) )
no_df = add_nans(no_df, ['hemoglobin', 'gender'])
no_df['label'] = 'No anemia'
no_df.head()

,hemoglobin,gender,ferritin,ret_count,segmented_neutrophils,tibc,mcv,label
0,13.947609,0,NaN,NaN,NaN,NaN,NaN,No anemia
1,16.943714,1,373.373386,0.364522,0.368434,443.030136,75.431234,No anemia
2,15.806368,1,281.333390,3.625152,4.324596,517.638263,97.032331,No anemia
3,15.113024,1,NaN,NaN,NaN,NaN,NaN,No anemia
4,12.811297,0,92.790118,3.016328,5.001375,245.380029,101.330348,No anemia


#### 1 - Vitamin B12/Folate deficiency anemia

In [10]:
vitamin_df = pd.DataFrame()
mcv_dict = get_feature_dict('mcv', anem_feat_arr)
neutrophils_dict = get_feature_dict('segmented_neutrophils', anem_feat_arr)
vitamin_df['hemoglobin'] = np.random.uniform(hb_dict['abnorm_lower'], 13, 10000)
vitamin_df['gender'] = np.random.choice([gender_dict['lower'], gender_dict['upper']], 10000)
vitamin_df.loc[vitamin_df.hemoglobin >= 12, 'gender'] = 1
vitamin_df['mcv'] = np.random.uniform(mcv_dict['upper'], mcv_dict['abnorm_upper'], 10000)
vitamin_df['segmented_neutrophils'] = np.random.uniform(neutrophils_dict['abnorm_lower'], neutrophils_dict['abnorm_upper'], 10000)
cols_to_populate = ['ferritin', 'ret_count', 'tibc']
vitamin_df = populate_columns(vitamin_df, cols_to_populate, anem_feat_arr, len(vitamin_df) )
vitamin_df = add_nans(vitamin_df, ['hemoglobin', 'gender', 'mcv', 'segmented_neutrophils'])
vitamin_df['label'] = 'Vitamin B12/Folate deficiency anemia'
vitamin_df.head()

,hemoglobin,gender,mcv,segmented_neutrophils,ferritin,ret_count,tibc,label
0,9.269562,0,104.090822,6.152398,NaN,NaN,NaN,Vitamin B12/Folate deficiency anemia
1,8.327591,1,100.726349,6.359490,263.485825,0.653504,326.684881,Vitamin B12/Folate deficiency anemia
2,11.164463,1,104.732319,2.710769,175.518475,NaN,358.462258,Vitamin B12/Folate deficiency anemia
3,6.857520,1,104.216121,1.698252,NaN,NaN,NaN,Vitamin B12/Folate deficiency anemia
4,11.251758,1,104.594266,1.422461,182.548318,0.449292,317.390831,Vitamin B12/Folate deficiency anemia


#### 2 - Unspecified anemia

In [11]:
unspec_df = pd.DataFrame()
unspec_df['hemoglobin'] = np.random.uniform(hb_dict['abnorm_lower'], 13, 10000)
unspec_df['gender'] = np.random.choice([gender_dict['lower'], gender_dict['upper']], 10000)
unspec_df.loc[unspec_df.hemoglobin >= 12, 'gender'] = 1
unspec_df['mcv'] = np.random.uniform(mcv_dict['upper'], mcv_dict['abnorm_upper'], 10000)
unspec_df['segmented_neutrophils'] = 0
cols_to_populate = ['ferritin', 'ret_count', 'tibc']
unspec_df = populate_columns(unspec_df, cols_to_populate, anem_feat_arr, len(unspec_df) )
unspec_df = add_nans(unspec_df, ['hemoglobin', 'gender', 'mcv', 'segmented_neutrophils'])
unspec_df['label'] = 'Unspecified anemia'
unspec_df.head()

,hemoglobin,gender,mcv,segmented_neutrophils,ferritin,ret_count,tibc,label
0,8.897282,1,100.186242,0,NaN,NaN,NaN,Unspecified anemia
1,6.387051,1,101.063104,0,219.772768,1.94141,429.747760,Unspecified anemia
2,10.407141,1,102.646372,0,42.283279,3.35404,300.573225,Unspecified anemia
3,7.477302,0,100.842741,0,NaN,NaN,NaN,Unspecified anemia
4,8.376418,1,101.236387,0,358.155498,3.13425,172.914617,Unspecified anemia


#### 3 - Anemia of chronic disease

In [12]:
acd_df = pd.DataFrame()
ferritin_dict = get_feature_dict('ferritin', anem_feat_arr)
tibc_dict = get_feature_dict('tibc', anem_feat_arr)
acd_df['hemoglobin'] = np.random.uniform(hb_dict['abnorm_lower'], 13, 10000)
acd_df['gender'] = np.random.choice([gender_dict['lower'], gender_dict['upper']], 10000)
acd_df.loc[acd_df.hemoglobin >= 12, 'gender'] = 1

acd_df['mcv'] = np.random.uniform(mcv_dict['abnorm_lower'], mcv_dict['lower'], 10000)
acd_df['ferritin'] = np.random.uniform(30, ferritin_dict['abnorm_upper'], 10000)

def define_tibc_acd(row):
    if row['ferritin']> 100:
        generated_num = np.random.choice([np.random.uniform(tibc_dict['abnorm_lower'], tibc_dict['abnorm_upper']), np.nan])
        return generated_num
    else:
        return np.random.uniform(tibc_dict['abnorm_lower'], tibc_dict['upper'])
acd_df['tibc'] = acd_df.apply(lambda row: define_tibc_acd(row), axis=1)
cols_to_populate = ['segmented_neutrophils', 'ret_count']
acd_df = populate_columns(acd_df, cols_to_populate, anem_feat_arr, len(acd_df) )
acd_df = add_nans(acd_df, ['hemoglobin', 'gender', 'mcv', 'ferritin', 'tibc'])
acd_df['label'] = 'Anemia of chronic disease'
acd_df.head()

,hemoglobin,gender,mcv,ferritin,tibc,segmented_neutrophils,ret_count,label
0,9.006692,1,76.412940,197.854151,430.375081,NaN,NaN,Anemia of chronic disease
1,11.215429,0,77.293383,460.495274,207.339958,1.453271,3.808731,Anemia of chronic disease
2,12.714183,1,75.496077,491.686175,NaN,4.751321,4.132620,Anemia of chronic disease
3,7.175200,1,77.234185,484.700078,NaN,NaN,NaN,Anemia of chronic disease
4,9.932120,1,76.015407,325.287898,NaN,4.646368,3.546949,Anemia of chronic disease


#### 4 - Iron deficiency anemia

In [13]:
ida_df = pd.DataFrame()
ida_df['hemoglobin'] = np.random.uniform(hb_dict['abnorm_lower'], 13, 10000)
ida_df['gender'] = np.random.choice([gender_dict['lower'], gender_dict['upper']], 10000)
ida_df.loc[ida_df.hemoglobin >= 12, 'gender'] = 1

ida_df['mcv'] = np.random.uniform(mcv_dict['abnorm_lower'], mcv_dict['lower'], 10000)
ida_df['ferritin'] = np.random.uniform(ferritin_dict['abnorm_lower'], 100, 10000)

def define_tibc_ida(row):
    if row['ferritin']< 30:
        generated_num = np.random.choice([np.random.uniform(tibc_dict['abnorm_lower'], tibc_dict['abnorm_upper']), np.nan])
        return generated_num
    else:
        return np.random.uniform(tibc_dict['upper'], tibc_dict['abnorm_upper'])
ida_df['tibc'] = ida_df.apply(lambda row: define_tibc_ida(row), axis=1)
cols_to_populate = ['segmented_neutrophils', 'ret_count']
ida_df = populate_columns(ida_df, cols_to_populate, anem_feat_arr, len(ida_df) )
ida_df = add_nans(ida_df, ['hemoglobin', 'gender', 'mcv', 'ferritin', 'tibc'])
ida_df['label'] = 'Iron deficiency anemia'
ida_df.head()

,hemoglobin,gender,mcv,ferritin,tibc,segmented_neutrophils,ret_count,label
0,8.739591,0,75.499873,10.223622,NaN,NaN,NaN,Iron deficiency anemia
1,12.308411,1,75.045870,27.010797,NaN,6.821118,1.120181,Iron deficiency anemia
2,7.329874,0,76.344396,23.971183,169.052418,4.316831,0.556877,Iron deficiency anemia
3,9.591604,0,76.303740,23.003296,201.826714,NaN,NaN,Iron deficiency anemia
4,9.157939,1,76.145714,9.571018,NaN,3.820393,1.102272,Iron deficiency anemia


#### 5 - Hemolytic anemia

In [14]:
hemolytic_df = pd.DataFrame()
ret_dict = get_feature_dict('ret_count', anem_feat_arr)
hemolytic_df['hemoglobin'] = np.random.uniform(hb_dict['abnorm_lower'], 13, 10000)
hemolytic_df['gender'] = np.random.choice([gender_dict['lower'], gender_dict['upper']], 10000)
hemolytic_df.loc[hemolytic_df.hemoglobin >= 12, 'gender'] = 1

hemolytic_df['mcv'] = np.random.uniform(mcv_dict['lower'], mcv_dict['upper'], 10000)
hemolytic_df['ret_count'] = np.random.uniform(2.1, ret_dict['abnorm_upper'], 10000)
cols_to_populate = ['ferritin', 'segmented_neutrophils', 'tibc']
hemolytic_df = populate_columns(hemolytic_df, cols_to_populate, anem_feat_arr, len(hemolytic_df) )
hemolytic_df = add_nans(hemolytic_df, ['hemoglobin', 'gender', 'mcv', 'ret_count'])
hemolytic_df['label'] = 'Hemolytic anemia'
hemolytic_df.head()

,hemoglobin,gender,mcv,ret_count,ferritin,segmented_neutrophils,tibc,label
0,12.793732,1,81.666105,2.957869,NaN,NaN,NaN,Hemolytic anemia
1,11.379106,1,99.461658,4.226354,230.198204,3.878443,367.377833,Hemolytic anemia
2,9.762204,1,88.048182,5.082087,467.824495,0.780075,484.973091,Hemolytic anemia
3,12.201397,1,91.315328,4.158392,NaN,NaN,NaN,Hemolytic anemia
4,12.875781,1,80.346447,4.558900,76.335966,5.588108,498.088040,Hemolytic anemia


#### 6 - Aplastic anemia

In [15]:
aplastic_df = pd.DataFrame()
aplastic_df['hemoglobin'] = np.random.uniform(hb_dict['abnorm_lower'], 13, 10000)
aplastic_df['gender'] = np.random.choice([gender_dict['lower'], gender_dict['upper']], 10000)
aplastic_df.loc[aplastic_df.hemoglobin >= 12, 'gender'] = 1

aplastic_df['mcv'] = np.random.uniform(mcv_dict['lower'], mcv_dict['upper'], 10000)
aplastic_df['ret_count'] = np.random.uniform(ret_dict['abnorm_lower'], 2, 10000)
cols_to_populate = ['ferritin', 'segmented_neutrophils', 'tibc']
aplastic_df = populate_columns(aplastic_df, cols_to_populate, anem_feat_arr, len(aplastic_df) )
aplastic_df = add_nans(aplastic_df, ['hemoglobin', 'gender', 'mcv', 'ret_count'])
aplastic_df['label'] = 'Aplastic anemia'
aplastic_df.head()

,hemoglobin,gender,mcv,ret_count,ferritin,segmented_neutrophils,tibc,label
0,10.671367,1,88.126494,1.518342,NaN,NaN,NaN,Aplastic anemia
1,9.124143,0,85.846038,1.921482,251.073623,2.191953,488.138603,Aplastic anemia
2,12.054159,1,96.104986,0.384968,NaN,1.133875,NaN,Aplastic anemia
3,11.763536,0,82.230693,1.589940,NaN,NaN,NaN,Aplastic anemia
4,11.071390,1,96.139974,0.426865,279.222554,1.420518,230.365854,Aplastic anemia


#### Altogether

In [16]:
total_anem_df = pd.concat([no_df, vitamin_df, unspec_df, acd_df, ida_df, hemolytic_df, aplastic_df], axis=0)
total_anem_df['new_label'] = total_anem_df.apply(lambda row: create_label(row), axis=1)
assert total_anem_df.label.tolist() == total_anem_df.new_label.tolist()
total_anem_df = total_anem_df.drop(['new_label'], axis=1)
total_anem_df = total_anem_df.sample(frac=1).reset_index(drop=True)
total_anem_df.head()

,hemoglobin,gender,ferritin,ret_count,segmented_neutrophils,tibc,mcv,label
0,11.875670,0,161.104070,3.981397,NaN,358.191297,77.370085,Anemia of chronic disease
1,9.296779,0,457.149834,1.874231,5.544915,NaN,78.717345,Anemia of chronic disease
2,9.780377,1,483.216550,4.041312,5.515052,304.155693,100.750651,Vitamin B12/Folate deficiency anemia
3,9.131688,0,57.578776,5.778568,1.976049,207.684776,96.855220,Hemolytic anemia
4,11.395680,0,260.627627,3.892581,1.320481,318.250045,102.206351,Vitamin B12/Folate deficiency anemia


#### Adding other features

In [17]:
add_feat_arr = [{'name':'serum_iron', 'lower':60, 'upper':170, 'abnorm_lower':20, 'abnorm_upper':250},
                {'name':'rbc', 'lower':3.8, 'upper':5.9, 'abnorm_lower': 2.5, 'abnorm_upper':7},
                {'name':'age', 'lower':20, 'upper':90, 'abnorm_lower':18, 'abnorm_upper':95},
                {'name':'indirect_bilirubin', 'lower':0.2, 'upper':1.2, 'abnorm_lower':0, 'abnorm_upper':3},
                {'name':'transferrin', 'lower':204, 'upper':360, 'abnorm_lower':150, 'abnorm_upper':450},
                {'name':'creatinine', 'lower':0.6, 'upper':1.3, 'abnorm_lower':0.2, 'abnorm_upper':2},
                {'name':'cholestrol', 'lower':30, 'upper':100, 'abnorm_lower':0, 'abnorm_upper':150},
                {'name':'copper', 'lower':62, 'upper':140, 'abnorm_lower':30, 'abnorm_upper':130},
                {'name':'ethanol', 'lower':0, 'upper':50, 'abnorm_lower':0, 'abnorm_upper':80},
                {'name':'folate', 'lower':2.7, 'upper':17, 'abnorm_lower':0.5, 'abnorm_upper':30},
                {'name':'glucose', 'lower':70, 'upper':100, 'abnorm_lower':40, 'abnorm_upper':140}
               ]

In [18]:
def create_dataset(sample_num, feat_arr, dist_type='uniform'):
    dataset = pd.DataFrame()
    for feat_dict in feat_arr:
        if feat_dict['name'] == 'gender':
            dataset[feat_dict['name']] = np.random.choice([feat_dict['lower'], feat_dict['upper']], sample_num)
        elif dist_type=='normal':
            dataset[feat_dict['name']] = normal_dist(dataset, feat_dict, sample_num)
        else:
            dataset[feat_dict['name']] = uniform_dist(dataset, feat_dict, sample_num)
    return dataset

In [19]:
add_ft_df = create_dataset(len(total_anem_df), add_feat_arr)
add_ft_df = add_nans(add_ft_df)
#add_ft_df = add_ft_df.fillna(-1)
add_ft_df.head()

,serum_iron,rbc,age,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose
0,NaN,6.918734,74.271409,NaN,441.651953,0.508604,86.888802,NaN,NaN,NaN,NaN
1,NaN,3.999210,70.112738,NaN,414.333999,0.784442,37.659905,NaN,NaN,NaN,NaN
2,118.428782,6.273777,87.682835,2.705387,350.319553,1.129411,140.917968,47.714319,62.722981,20.039071,46.973234
3,NaN,3.773108,39.425050,NaN,353.517996,1.569724,26.393979,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
total_df = pd.concat([total_anem_df, add_ft_df], axis=1)
total_df = total_df[['hemoglobin', 'ferritin', 'ret_count', 'segmented_neutrophils', 'tibc', 'mcv', 'serum_iron', 
                     'rbc', 'age', 'gender', 'indirect_bilirubin', 'transferrin', 'creatinine', 'cholestrol', 
                     'copper', 'ethanol', 'folate', 'glucose', 'label']]
total_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,indirect_bilirubin,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label
0,11.875670,161.104070,3.981397,NaN,358.191297,77.370085,NaN,6.918734,74.271409,0,NaN,441.651953,0.508604,86.888802,NaN,NaN,NaN,NaN,Anemia of chronic disease
1,9.296779,457.149834,1.874231,5.544915,NaN,78.717345,NaN,3.999210,70.112738,0,NaN,414.333999,0.784442,37.659905,NaN,NaN,NaN,NaN,Anemia of chronic disease
2,9.780377,483.216550,4.041312,5.515052,304.155693,100.750651,118.428782,6.273777,87.682835,1,2.705387,350.319553,1.129411,140.917968,47.714319,62.722981,20.039071,46.973234,Vitamin B12/Folate deficiency anemia
3,9.131688,57.578776,5.778568,1.976049,207.684776,96.855220,NaN,3.773108,39.425050,0,NaN,353.517996,1.569724,26.393979,NaN,NaN,NaN,NaN,Hemolytic anemia
4,11.395680,260.627627,3.892581,1.320481,318.250045,102.206351,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vitamin B12/Folate deficiency anemia


In [21]:
# total_df.to_csv('../../final/data/dataset_by_type.csv', index=False)

#### 7 - Inconclusive diagnosis

In [22]:
col_list = [col for col in total_df.columns if col not in ['hemoglobin', 'gender', 'age', 'mcv', 'label']]

In [23]:
nan_frac = 0.1
trial_df = utils.generate_nans(total_df.copy(), col_list, nan_frac)

In [24]:
trial_df.label.value_counts()

Anemia of chronic disease               10000
Vitamin B12/Folate deficiency anemia    10000
Hemolytic anemia                        10000
Aplastic anemia                         10000
No anemia                               10000
Unspecified anemia                      10000
Iron deficiency anemia                  10000
Name: label, dtype: int64

In [25]:
trial_df['label'] = trial_df.apply(lambda row: create_label(row), axis=1)
trial_df.label.value_counts()

No anemia                               10000
Aplastic anemia                          9029
Hemolytic anemia                         9025
Vitamin B12/Folate deficiency anemia     9004
Unspecified anemia                       8964
Anemia of chronic disease                8859
Iron deficiency anemia                   8398
Inconclusive diagnosis                   6721
Name: label, dtype: int64

In [26]:
# trial_df.to_csv(f'../../final/data/dataset_with_inconclusive.csv', index=False)

#### Adding derived features

In [27]:
trial_df['hematocrit'] = 3*trial_df['hemoglobin']
trial_df['rbc'] = (30*trial_df['hemoglobin'])/trial_df['mcv']
trial_df['tsat'] = (trial_df['serum_iron']/trial_df['tibc'])*100
trial_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,age,gender,...,transferrin,creatinine,cholestrol,copper,ethanol,folate,glucose,label,hematocrit,tsat
0,11.875670,161.104070,3.981397,NaN,358.191297,77.370085,NaN,4.604753,74.271409,0,...,441.651953,0.508604,86.888802,NaN,NaN,NaN,NaN,Anemia of chronic disease,35.627010,NaN
1,9.296779,457.149834,1.874231,5.544915,NaN,78.717345,NaN,3.543099,70.112738,0,...,414.333999,0.784442,37.659905,NaN,NaN,NaN,NaN,Anemia of chronic disease,27.890337,NaN
2,9.780377,483.216550,4.041312,5.515052,304.155693,100.750651,118.428782,2.912252,87.682835,1,...,350.319553,1.129411,140.917968,47.714319,62.722981,20.039071,46.973234,Vitamin B12/Folate deficiency anemia,29.341132,38.936895
3,9.131688,57.578776,5.778568,1.976049,207.684776,96.855220,NaN,2.828455,39.425050,0,...,353.517996,1.569724,26.393979,NaN,NaN,NaN,NaN,Hemolytic anemia,27.395064,NaN
4,11.395680,NaN,NaN,1.320481,318.250045,102.206351,NaN,3.344904,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vitamin B12/Folate deficiency anemia,34.187041,NaN


In [28]:
cols_to_drop = ['transferrin', 'age', 'indirect_bilirubin']
trial_df = trial_df.drop(cols_to_drop, axis=1)
trial_df = trial_df[[c for c in trial_df if c != 'label'] + ['label']]
trial_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,11.875670,161.104070,3.981397,NaN,358.191297,77.370085,NaN,4.604753,0,0.508604,86.888802,NaN,NaN,NaN,NaN,35.627010,NaN,Anemia of chronic disease
1,9.296779,457.149834,1.874231,5.544915,NaN,78.717345,NaN,3.543099,0,0.784442,37.659905,NaN,NaN,NaN,NaN,27.890337,NaN,Anemia of chronic disease
2,9.780377,483.216550,4.041312,5.515052,304.155693,100.750651,118.428782,2.912252,1,1.129411,140.917968,47.714319,62.722981,20.039071,46.973234,29.341132,38.936895,Vitamin B12/Folate deficiency anemia
3,9.131688,57.578776,5.778568,1.976049,207.684776,96.855220,NaN,2.828455,0,1.569724,26.393979,NaN,NaN,NaN,NaN,27.395064,NaN,Hemolytic anemia
4,11.395680,NaN,NaN,1.320481,318.250045,102.206351,NaN,3.344904,0,NaN,NaN,NaN,NaN,NaN,NaN,34.187041,NaN,Vitamin B12/Folate deficiency anemia


In [ ]:
#df.to_csv('../../data/full_anemia_dataset.csv', index=False)